In [1]:
from pyspark.sql import SparkSession

# Spark сессия с максимальной производительностью

In [ ]:
spark = (SparkSession.builder
         .appName("Full Power")
         .master("spark://spark-master:7077")
         .config("spark.executor.instances", "2")  
         .config("spark.executor.cores", "10")     
         .config("spark.executor.memory", "16g")   
         .config("spark.driver.memory", "4g")      
         .getOrCreate()
        )

In [ ]:
df = spark.range(10)
df.show()
spark.stop()

```python
# Создание SparkSession с настройками для максимальной производительности
spark = (SparkSession.builder
         # Имя приложения для идентификации в Spark UI и логах
         .appName("Full Power")
         
         # Подключение к Spark кластеру
         # spark-master:7077 - главный узел кластера, который управляет распределенными вычислениями
         .master("spark://spark-master:7077")
         
         # КОНФИГУРАЦИЯ ИСПОЛНИТЕЛЕЙ (EXECUTORS)
         # Executor - процесс, выполняющий задачи на рабочих узлах кластера
         
         # Количество исполнителей на рабочем узле
         # 2 исполнителя будут запущены на каждом worker node
         # Это позволяет лучше изолировать задачи и управлять памятью
         .config("spark.executor.instances", "2")  
         
         # Количество CPU ядер на каждый исполнитель
         # 10 ядер позволяют выполнять до 10 параллельных задач в каждом executor
         # Оптимально для узлов с большим количеством процессорных ядер
         .config("spark.executor.cores", "10")     
         
         # Объем оперативной памяти на каждый исполнитель
         # 16GB RAM для каждого executor процесса
         # Включает память для выполнения задач и хранения данных в памяти
         # Структура памяти executor:
         # - Execution Memory (шаффлинг, joins, агрегации)
         # - Storage Memory (кэшированные DataFrame/RDD)
         # - User Memory (пользовательские структуры данных)
         # - Reserved Memory (системные нужды Spark ~300MB)
         .config("spark.executor.memory", "16g")   
         
         # КОНФИГУРАЦИЯ ДРАЙВЕРА
         # Driver - главный процесс, координирующий выполнение приложения
         
         # Память драйвера - 4GB для:
         # - Хранения метаданных приложения
         # - Сбора результатов (collect())
         # - Broadcast переменных
         # - Управления задачами и планирования
         .config("spark.driver.memory", "4g")      
         
         # Создание или получение существующей сессии
         .getOrCreate()
        )
```

## Детальное объяснение архитектуры:

### **Распределение ресурсов:**
```
Worker Node (физическая/виртуальная машина)
├── Executor 1 (16GB RAM, 10 cores)
│   ├── Task Slot 1
│   ├── Task Slot 2
│   └── ... Task Slot 10
└── Executor 2 (16GB RAM, 10 cores)
    ├── Task Slot 1
    ├── Task Slot 2
    └── ... Task Slot 10
```

### **Производительность и масштабирование:**

1. **Параллелизм задач:**
   - Всего слотов для задач: `2 executors × 10 cores = 20 параллельных задач`
   - Это позволяет обрабатывать до 20 партиций данных одновременно

2. **Распределение памяти:**
   ```python
   # Примерная структура памяти executor (16GB):
   - Spark Memory: ~60% (9.6GB)
     * Storage Memory: кэшированные данные
     * Execution Memory: операции шаффлинга
   - User Memory: ~25% (4GB) - пользовательские структуры
   - Reserved: ~5% (0.8GB) - системные нужды
   ```

3. **Рекомендации для продакшена:**
   ```python
   # Дополнительные оптимизации для "Full Power":
   .config("spark.sql.adaptive.enabled", "true")          # Адаптивное выполнение запросов
   .config("spark.sql.adaptive.coalescePartitions.enabled", "true") # Объединение партиций
   .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") # Быстрая сериализация
   .config("spark.sql.shuffle.partitions", "200")         # Партиции для шаффлинга
   .config("spark.default.parallelism", "40")             # Параллелизм по умолчанию
   ```

### **Типичные сценарии использования:**
- **Обработка больших данных** (сотни GB - TB)
- **Сложные ETL-пайплайны** с множеством преобразований
- **Машинное обучение на больших датасетах**
- **Аналитика в реальном времени** с высокими требованиями к производительности

Такая конфигурация подходит для мощных продакшен-кластеров с значительными вычислительными ресурсами.

Отличная конфигурация! С вашим железом и текущим setup вы можете обрабатывать значительные объемы данных. Давайте разберем детально:

## **Текущие ресурсы кластера:**

### Вычислительная мощность:
```
Всего в кластере:
- 2 worker × 10 cores = 20 CPU ядер
- 2 worker × 18GB RAM = 36GB оперативной памяти
```

### Память для обработки данных:
```python
# Доступная память для данных (приблизительно):
Общая RAM: 36GB
- Системные нужды Spark: ~2GB
- Резерв: ~4GB
= Доступно для данных: ~30GB
```

## **Объемы данных для обработки:**

### **1. Оптимальные объемы (комфортная работа):**
- **В памяти:** 15-25 GB данных
- **На диске (с шаффлингом):** 50-100 GB данных
- **Ежедневная обработка:** 200-500 GB (с разбивкой на батчи)

### **2. Максимальные объемы (с оптимизацией):**
- **Единоразово в памяти:** до 30 GB
- **С промежуточной записью на диск:** 200-300 GB
- **Ежедневно:** 1-2 TB (с правильным партиционированием)

## **Рекомендации по оптимизации:**

### Конфигурация Spark для вашего железа:
```python
spark = (SparkSession.builder
    .appName("Optimized for i5-14600K")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "2")  
    .config("spark.executor.cores", "8")      # Оставляем 2 ядра на систему
    .config("spark.executor.memory", "14g")   # Оставляем 4GB на систему
    .config("spark.driver.memory", "8g")      # Увеличиваем для больших collect()
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128MB")
    .config("spark.sql.shuffle.partitions", "100")  # 8 cores × 2 workers × 6 = 96
    .config("spark.default.parallelism", "96")
    .config("spark.memory.fraction", "0.8")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)
```

## **Типичные сценарии обработки:**

### **1. Аналитические запросы:**
```python
# Мгновенная обработка: 5-10 GB
# Запросы до 1 минуты: 10-20 GB  
# Длительные ETL: 50-100 GB за запуск
```

### **2. Машинное обучение:**
```python
# Обучение моделей:
# - До 10M строк × 100 фич: комфортно
# - До 50M строк × 50 фич: с оптимизацией
# - Feature engineering: 20-30 GB наборов данных
```

### **3. Стриминг данных:**
```python
# Потоковая обработка:
# - До 50,000 событий/секунду
# - Микробатчи по 10-30 секунд
# - Окна агрегации: 1-60 минут
```

## **Мониторинг и ограничения:**

### Признаки нехватки ресурсов:
```python
# Тревожные сигналы:
- Frequent GC pauses (> 10% времени)
- Spill на диск (Disk spill bytes > 0)
- OOM errors
- Tasks taking > 5-10 minutes
```

### Практические примеры объемов:
```
✅ Легко:     Анализ 10GB CSV (~100M строк)
✅ Комфортно: ETL 50GB данных ежедневно  
⚠️ Возможно:  Обработка 150GB с оптимизацией
❌ Сложно:    Единовременная обработка >200GB
```

## **Рекомендации для вашего setup:**

1. **Используйте партиционирование** для больших datasets
2. **Кэшируйте часто используемые DataFrames**
3. **Настройте правильные типы данных** (меньше памяти)
4. **Используйте формат Parquet/ORC** вместо CSV/JSON
5. **Мониторьте Spark UI** на портах 8085-8087

Ваша система отлично подходит для серьезных data engineering задач и может обрабатывать гигабайты данных ежедневно!

# Коннектор к `MinIO`

In [ ]:
spark = (SparkSession.builder
         .appName("MinIO Test")
         .master("spark://spark-master:7077") 
         .config("spark.jars.packages", 
                 "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")     
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
         .getOrCreate()
        )

In [ ]:
df = spark.range(10)
df.write.mode("overwrite").csv("s3a://learn-bucket/test-csv-33")
print("✅ Успешно!")

In [ ]:
spark.stop()

```python
# Создание SparkSession - основной точки входа для работы с Spark
spark = (SparkSession.builder
         # Указываем имя приложения - будет отображаться в Spark UI
         .appName("MinIO Test")
         
         # Задаем URL мастер-ноды Spark кластера
         # spark://spark-master:7077 - означает, что Spark работает в кластерном режиме
         # spark-master - имя хоста мастер-ноды (обычно задается в Docker Compose или Kubernetes)
         # 7077 - стандартный порт для подключения драйверов к кластеру
         .master("spark://spark-master:7077") 
         
         # Указываем необходимые JAR-пакеты для работы с S3-совместимыми хранилищами
         # hadoop-aws:3.3.4 - библиотека Hadoop для работы с AWS S3
         # aws-java-sdk-bundle:1.12.262 - AWS Java SDK для клиентских операций
         # Spark автоматически скачает эти зависимости при запуске
         .config("spark.jars.packages", 
                 "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")     
         
         # Настройка доступа к MinIO - логин (access key)
         # MinIO по умолчанию использует minioadmin/minioadmin
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
         
         # Настройка доступа к MinIO - пароль (secret key)
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
         
         # Указываем endpoint MinIO сервера
         # http://minio:9000 - MinIO работает по HTTP на порту 9000
         # 'minio' - имя сервиса в Docker-сети
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         
         # Включаем path-style доступ к бакетам (требуется для MinIO)
         # В отличие от virtual-hosted style, где бакет в домене: bucket.host.com
         # Path-style: host.com/bucket/path/to/file
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         
         # Указываем реализацию файловой системы для S3
         # S3AFileSystem - Hadoop файловая система для работы с S3-совместимыми хранилищами
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         
         # Отключаем SSL/TLS шифрование соединения
         # Так как мы используем HTTP (не HTTPS) для подключения к MinIO
         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
         
         # Создаем или получаем существующий SparkSession
         # getOrCreate() предотвращает создание дублирующихся сессий
         .getOrCreate()
        )
```

## Ключевые моменты конфигурации:

### 1. **Архитектура подключения**
- Spark драйвер подключается к кластеру через мастер-ноду
- Executor'ы в кластере получат те же конфигурации S3/MinIO

### 2. **Работа с MinIO**
- MinIO эмулирует S3 API, поэтому используем S3A connector
- Path-style access обязателен для MinIO
- Отключен SSL т.к. используется HTTP

### 3. **Зависимости**
- `hadoop-aws` предоставляет S3A файловую систему  
- `aws-java-sdk` обеспечивает низкоуровневые S3 операции

### 4. **Безопасность в продакшене**
```python
# В реальных сценариях лучше использовать:
.config("spark.hadoop.fs.s3a.access.key", os.environ.get("AWS_ACCESS_KEY"))
.config("spark.hadoop.fs.s3a.secret.key", os.environ.get("AWS_SECRET_KEY"))
```

После создания сессии можно работать с MinIO как с обычной S3 файловой системой:
```python
df = spark.read.parquet("s3a://my-bucket/data/")
```